In [2]:
class Graph():
    #初始化基本结构
    def __init__(self, n):
        self.vNum = n  
        self.vexs = [0] * self.vNum ##建立一个全为0的顶点矩阵
        self.arcs = [[float('inf')] * self.vNum for _ in range(self.vNum)] #建立一个全为正无穷的邻接矩阵
        for i in range(n):
            self.arcs[i][i] = 0 #对角线为0
    #添加顶点
    def addVex(self, v, i):
        self.vexs[i] = v
    #添加边，并设置权
    def addArcs(self, v1, v2, cost):
        self.arcs[v1][v2] = cost
        self.arcs[v2][v1] = cost
        
def dijkstra(G, v0):
    INF = float('inf')
    S = [v0] #已找到最短路径的顶点集合
    U = [v for v in range(G.vNum) if v not in S] #记录还未确定最短路径的顶点集合
    D = [INF] * G.vNum #存在放从顶点v0到v的最短路径长度
    D[v0] = 0 #起点到自己的距离为0
    P = [''] * G.vNum #存放从v0到v最短路径上的顶点序列
    while len(S) < G.vNum: #当已找到最短路径的顶点小于n时
        min_value = INF
        col = -1
        row = -1
        for s in S: #以已找到最短路径的顶点所在行为搜索对象
            for u in U: #从U中搜索尚未记录的顶点
                if G.arcs[s][u] + D[s] < min_value: #找出最小值
                    min_value = G.arcs[s][u] + D[s] 
                    #记录所在行列
                    row = s         
                    col = u
                    if P[s] == '':
                        P[u] = str(G.vexs[s]) + '-' + str(G.vexs[u])
                    else:
                        P[u] = str(P[s]) + '-' + str(G.vexs[u])
        if col == -1 or row == -1: #若没找出最小值且顶点还未找完，说明图中存在不连通的顶点
            break
        S.append(col) #在S中添加已找到的顶点
        U.remove(col) #从U中移除已找到的顶点
        D[col] = min_value #v0到该顶点的最短距离即为min_value  
    P[v0] = str(G.vexs[v0])
    return D, P

In [3]:
#测试
n = 5
#创建一个图
g = Graph(n)
#往图中添加顶点
g.addVex('A', 0)
g.addVex('B', 1)
g.addVex('C', 2)
g.addVex('D', 3)
g.addVex('E', 4)
#往图中添加边及权
g.addArcs(0, 1, 60)
g.addArcs(0, 2, 80)
g.addArcs(0, 3, 30)
g.addArcs(1, 2, 40)
g.addArcs(1, 3, 75)
g.addArcs(2, 4, 35)
g.addArcs(3, 4, 45)
distance, route = dijkstra(g, 0)
for i in range(len(distance)):
    print(f'从A到{g.vexs[i]}的最短路径为{route[i]},距离为{distance[i]}')

从A到A的最短路径为A,距离为0
从A到B的最短路径为A-B,距离为60
从A到C的最短路径为A-C,距离为80
从A到D的最短路径为A-D,距离为30
从A到E的最短路径为A-D-E,距离为75


In [5]:
import json
#读取geojson数据
file_path = '/Users/jocelynli/Library/CloudStorage/OneDrive-stu.ecnu.edu.cn/ECNU/大三上/数据结构与算法/project/第12周/road-sample-utm-fixed.geojson'
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)
roads = data['features']
Vexes = {} #存放每条路的端点，key为端点的坐标，value为0或1，0代表该点未被添加到图中，1代表该点已被添加到图中
for road in roads: 
    coordinates = road['geometry']['coordinates'][0]
    start = tuple(coordinates[0]) #将端点的坐标从列表转化为元组，因为字典的键必须是可哈希的类型
    end = tuple(coordinates[-1])
    #将每条路的两个端点存放到字典中，避免重复存放同一个点
    if start not in Vexes:
        Vexes[start] = 0 #初始设定该点未被添加到图中
    if end not in Vexes:
        Vexes[end] = 0
keys_list = list(Vexes.keys()) #将字典中的key作为列表，便于获取序号
G = Graph(len(Vexes)) #创建图
for road in roads: #将每条路的两个端点作为图的顶点
    road_length = road['properties']['SHAPE_Leng']
    coordinates = road['geometry']['coordinates'][0]
    start = tuple(coordinates[0]) 
    if Vexes[start] == 0: #若该点未被添加到图中，则向图中添加该点
        start_idx = keys_list.index(start) 
        G.addVex(start, start_idx)
        Vexes[start] = 1 #更新value，该点已被添加到图中
    else:
        start_idx = keys_list.index(start)
    end = tuple(coordinates[-1])
    if Vexes[end] == 0:
        end_idx = keys_list.index(end)
        G.addVex(end, end_idx)
        Vexes[end] = 1 
    else:
        end_idx = keys_list.index(end)
    G.addArcs(start_idx, end_idx, road_length) #往图中添加边及权

#设定要查找的两个点
start_point = (-1361.7764848724764, -24537.437368433864)
end_point = (-2477.243109911360079, -21460.279477568037692)
index1 = keys_list.index(start_point)
index2 = keys_list.index(end_point)
Distance, P = dijkstra(G, index1)
print(f'点{start_point}到点{end_point}的最短距离为：{Distance[index2]}米')

点(-1361.7764848724764, -24537.437368433864)到点(-2477.24310991136, -21460.279477568038)的最短距离为：3530.62米
